In [2]:
import duckdb
con = duckdb.connect()

In [3]:
con.sql("LOAD postgres;")

In [4]:
con.sql("ATTACH 'host=localhost port=5432 dbname=postgres user=postgres password=postgres' AS postgres (TYPE postgres);")

### Populando PIT_Customers

In [23]:
con.sql("""
    INSERT into postgres.silver_business_vault.PIT_Customers   
    select 
        h.Hash_Key_Customer,
        current_date as PIT_Dt_Load,
        sc.Dt_Load as Name_Dt_Load,
        sca.Dt_Load as Address_Dt_Load
    from postgres.silver_raw_vault.HUB_Customers h
    inner join postgres.silver_raw_vault.SAT_Customer sc on h.Hash_Key_Customer = sc.Hash_Key_Customer 
    inner join postgres.silver_raw_vault.SAT_Customer_Address sca on h.Hash_Key_Customer = sca.Hash_Key_Customer
""")


In [24]:
con.sql("select * from postgres.silver_business_vault.PIT_Customers")

┌──────────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│        Hash_Key_Customer         │     PIT_Dt_Load     │    Name_Dt_Load     │   Adress_Dt_Load    │
│             varchar              │      timestamp      │      timestamp      │      timestamp      │
├──────────────────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ 3843d32087bb0e88d4a71d4822dfcf08 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ 936a02d4ffa95853a9185c2f65dff746 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ 5af7cc596883d1058d430ba55e9efe15 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ 8f02a7d7eed9a7ce2bf93793de025fba │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ d87fe20078f7e4a277e16c5faef6f592 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ 7259235dfc93163e32b405fa5f7ebbd4 │ 2024-04-23 00:00:00 │ 2024-04-23 00:

### Populando PIT_Employees

In [25]:
con.sql("""
        INSERT into postgres.silver_business_vault.PIT_Employee  
        select 
        	h.Hash_Key_Employee ,
        	current_date as PIT_Dt_Load,
        	sc.Dt_Load as Name_Dt_Load,
        	sca.Dt_Load as Address_Dt_Load
        from postgres.silver_raw_vault.HUB_Employee h
        inner join postgres.silver_raw_vault.SAT_Employee sc on h.Hash_Key_Employee = sc.Hash_Key_Employee
        inner join postgres.silver_raw_vault.SAT_Employee_Address sca on h.Hash_Key_Employee = sca.Hash_Key_Employee
""")

In [26]:
con.sql("select * from postgres.silver_business_vault.PIT_Employee")

┌──────────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│        Hash_Key_Employee         │     PIT_Dt_Load     │    Name_Dt_Load     │   Adress_Dt_Load    │
│             varchar              │      timestamp      │      timestamp      │      timestamp      │
├──────────────────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ c4ca4238a0b923820dcc509a6f75849b │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ c81e728d9d4c2f636f067f89cc14862c │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ eccbc87e4b5ce2fe28308fd9f2a7baf3 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ a87ff679a2f3e71d9181a67b7542122c │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ e4da3b7fbbce2345d7772b0674a318d5 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ 1679091c5a880faf6fb5e6087eb1b2dc │ 2024-04-23 00:00:00 │ 2024-04-23 00:

### Populando Bridge_Store

In [11]:
con.sql("""
        INSERT into postgres.silver_business_vault.Bridge_Store 
        select
            md5(cast(o.Order_ID as varchar)||cast(e.Employee_ID as varchar)||c.Customer_ID||cast(p.Product_ID as varchar)||cast(s.Shipper_ID as varchar)) as Hash_Key_Bridge
        	,current_date as Bridge_Dt_Load
        	,o.Hash_Key_Order
            ,o.Order_ID
            ,e.Hash_Key_Employee
        	,e.Employee_ID
        	,c.Hash_Key_Customer
        	,c.Customer_ID
        	,p.Hash_Key_Product
        	,p.Product_ID
        	,s.Hash_Key_Shipper
        	,s.Shipper_ID
        	,lpod.Hash_Key_Link_P_O
        	,lso.Hash_Key_Link_S_O
        	,leo.Hash_Key_Link_E_O
        	,lco.Hash_Key_Link_C_O
        
        from postgres.silver_raw_vault.HUB_Orders o
        inner join postgres.silver_raw_vault.SAT_Orders so on o.Hash_Key_Order = so.Hash_Key_Order 
        inner join postgres.silver_raw_vault.LINK_Products_Orders_Detail lpod on o.Hash_Key_Order = lpod.Hash_Key_Orders 
        inner join postgres.silver_raw_vault.SAT_Orders_Details sod on lpod.Hash_Key_Link_P_O  = sod.Hash_Key_Order_P_O
        inner join postgres.silver_raw_vault.HUB_Products p on lpod.Hash_Key_Product = p.Hash_Key_Product 
        inner join postgres.silver_raw_vault.SAT_Product sp on p.Hash_Key_Product = sp.Hash_Key_Product 
        inner join postgres.silver_raw_vault.LINK_Employee_Orders leo on o.Hash_Key_Order = leo.Hash_Key_Orders 
        inner join postgres.silver_raw_vault.HUB_Employee e on leo.Hash_Key_Employee = e.Hash_Key_Employee 
        inner join postgres.silver_raw_vault.SAT_Employee se on e.Hash_Key_Employee = se.Hash_Key_Employee 
        inner join postgres.silver_raw_vault.SAT_Employee_Address sea on e.Hash_Key_Employee = sea.Hash_Key_Employee 
        inner join postgres.silver_raw_vault.LINK_Customers_Orders lco on o.Hash_Key_Order = lco.Hash_Key_Orders 
        inner join postgres.silver_raw_vault.HUB_Customers c on lco.Hash_Key_Customer = c.Hash_Key_Customer 
        inner join postgres.silver_raw_vault.SAT_Customer sc on c.Hash_Key_Customer = sc.Hash_Key_Customer
        inner join postgres.silver_raw_vault.SAT_Customer_Address sca on c.Hash_Key_Customer = sca.Hash_Key_Customer 
        inner join postgres.silver_raw_vault.LINK_Shipper_Orders lso on o.Hash_Key_Order = lso.Hash_Key_Orders
        inner join postgres.silver_raw_vault.HUB_Shippers s on lso.Hash_Key_Shipper = s.Hash_Key_Shipper
        inner join postgres.silver_raw_vault.SAT_Shipper ss on ss.Hash_Key_Shipper = s.Hash_Key_Shipper 
""")

In [13]:
con.sql("select * from postgres.silver_business_vault.Bridge_Store")

┌──────────────────────┬─────────────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┐
│   Hash_Key_Bridge    │   Bridge_Dt_Load    │   Hash_Key_Orders    │ … │ Hash_Key_Link_Empl…  │ Hash_Key_Link_Cust…  │
│       varchar        │      timestamp      │       varchar        │   │       varchar        │       varchar        │
├──────────────────────┼─────────────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┤
│ 40aa95052e5140ea82…  │ 2024-04-24 00:00:00 │ cfdf55d9c4687ca7b7…  │ … │ 0a1595cacaa6446fb5…  │ ab5b727c74d99fa5d1…  │
│ c07b0bca73d3e70292…  │ 2024-04-24 00:00:00 │ cfdf55d9c4687ca7b7…  │ … │ 0a1595cacaa6446fb5…  │ ab5b727c74d99fa5d1…  │
│ 2266788f1c9568c265…  │ 2024-04-24 00:00:00 │ cfdf55d9c4687ca7b7…  │ … │ 0a1595cacaa6446fb5…  │ ab5b727c74d99fa5d1…  │
│ bd083a2963ae0e8a16…  │ 2024-04-24 00:00:00 │ 28fda3e1d182e68d52…  │ … │ f4ea93b7fcbb667a17…  │ 71e16c25bd58276d8c…  │
│ f22b53421ec2e4f203…  │ 2024-04-24 00:0

In [14]:
con.close()